In [1]:
%matplotlib inline

Diferenciación automática con ``torch.autograd``
 =======================================

 Al entrenar redes neuronales, el algoritmo más utilizado es
 **propagación hacia atrás**.  En este algoritmo, los parámetros (pesos del modelo) son
 ajustado de acuerdo con el **gradiente** de la función de pérdida con respecto
 al parámetro dado.

 Para calcular esos gradientes, PyTorch tiene un motor de diferenciación incorporado
 llamado ``torch.autograd``.  Es compatible con el cálculo automático de pendiente para cualquier
 gráfico computacional.

 Considere la red neuronal de una capa más simple, con entrada ``x``,
 parámetros ``w`` y ``b``, y alguna función de pérdida.  Se puede definir en
 PyTorch de la siguiente manera:

In [2]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

Tensores, Funciones y Gráfico computacional
------------------------------------------

Este código define el siguiente **gráfico computacional**:

.. figure:: /_static/img/basics/comp-graph.png
   :alt:

En esta red, ``w`` y ``b`` son **parámetros**, que necesitamos
optimizar. Por lo tanto, necesitamos poder calcular los gradientes de pérdida
función con respecto a esas variables. Para ello, establecemos
la propiedad ``requires_grad`` de esos tensores.


<div class="alert alert-info"><h4>Nota</h4><p>Puede establecer el valor de ``requires_grad`` al crear un
           tensor, o posterior usando el método ``x.requires_grad_(True)``.p></div>



Una función que aplicamos a los tensores para construir un gráfico computacional es
de hecho un objeto de la clase ``Function``. Este objeto sabe cómo
calcular la función en la dirección *hacia adelante*, y también cómo calcular
su derivada durante el paso de *propagación hacia atrás*. Una referencia a
la función de propagación hacia atrás se almacena en la propiedad ``grad_fn`` de un
tensor. Puede encontrar más información de ``Function`` `en el
documentación <https://pytorch.org/docs/stable/autograd.html#function>`__.


In [4]:
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fb334603a90>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fb334603910>


Computing Gradients
-------------------

Para optimizar los pesos de los parámetros en la red neuronal, necesitamos
calcular las derivadas de nuestra función de pérdida con respecto a los parámetros,
es decir, necesitamos $\frac{\partial loss}{\partial w}$ and
$\frac{\partial pérdida}{\partial b}$ bajo unos valores fijos de
``x`` y ``y``. Para calcular esas derivadas, llamamos
``loss.backward()``, y luego recuperar los valores de ``w.grad`` y
``b.grad``:




In [5]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2421, 0.2549, 0.1080],
        [0.2421, 0.2549, 0.1080],
        [0.2421, 0.2549, 0.1080],
        [0.2421, 0.2549, 0.1080],
        [0.2421, 0.2549, 0.1080]])
tensor([0.2421, 0.2549, 0.1080])


<div class="alert alert-info"><h4>Nota</h4><p>- Solo podemos obtener las propiedades ``grad`` para la hoja
     nodos del grafo computacional, que tienen la propiedad ``requires_grad``
     establecido en ``True``. Para todos los demás nodos en nuestro gráfico, los gradientes no serán
     disponible.
    - Solo podemos realizar cálculos de gradiente usando
    ``backward`` una vez en un gráfico dado, por motivos de rendimiento. si necesitamos
     hacer varios``backward`` llamadas en el mismo gráfico, tenemos que pasar
     ``retain_graph=True`` al llamar ``backward``.</p></div>




Deshabilitar el seguimiento de degradado
---------------------------

Por defecto, todos los tensores con ``requires_grad=True`` siguen su
historial computacional y cálculo de gradiente de soporte. Sin embargo, hay
hay algunos casos en los que no necesitamos hacer eso, por ejemplo, cuando tenemos
entrenamos el modelo y solo queremos aplicarlo a algunos datos de entrada, es decir,
solo quiere hacer cálculos *hacia adelante* a través de la red. Nosotros podemos parar
seguimiento de cálculos rodeando nuestro código de cálculo con
Bloque ``torch.no_grad()``:


In [6]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


Otra forma de lograr el mismo resultado es usar el método ``detach()``
 en el tensor:

In [7]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


Hay razones por las que es posible que desee deshabilitar el seguimiento de gradiente:
   - Para marcar algunos parámetros en su red neuronal como **parámetros congelados**.  Esto es
     un escenario muy común para
     `ajuste fino de una red preentrenada <https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html>`__
   - Para **acelerar los cómputos** cuando solo está haciendo pases hacia adelante, porque los cómputos en tensores que sí lo hacen
     no rastrear gradientes sería más eficiente.



Más sobre gráficos computacionales
 ----------------------------
 Conceptualmente, autograd mantiene un registro de datos (tensores) y todos los ejecutados
 operaciones (junto con los nuevos tensores resultantes) en un acíclico dirigido
 gráfico (DAG) que consta de
 `Función <https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function>`__
 objetos.  En este DAG, las hojas son los tensores de entrada, las raíces son la salida
 tensores.  Al trazar este gráfico desde las raíces hasta las hojas, puedes
 calcular automáticamente los gradientes utilizando la regla de la cadena.

 En un pase hacia adelante, autograd hace dos cosas simultáneamente:

 - ejecutar la operación solicitada para calcular un tensor resultante
 - mantener la *función de gradiente* de la operación en el DAG.

 El pase hacia atrás comienza cuando se llama ``.backward()`` en el DAG
 raíz.  ``autograd`` entonces:

 - calcula los gradientes de cada ``.grad_fn``,
 - los acumula en el atributo ``.grad`` del tensor respectivo
 - utilizando la regla de la cadena, se propaga hasta los tensores de hoja.

 <div class="alert alert-info"><h4>Nota</h4><p>**Los DAG son dinámicos en PyTorch**
   Una cosa importante a tener en cuenta es que el gráfico se recrea desde cero;  después de cada
   llamada ``.backward()``, autograd comienza a llenar un nuevo gráfico.  Esto es
   exactamente lo que le permite usar declaraciones de flujo de control en su modelo;
   puede cambiar la forma, el tamaño y las operaciones en cada iteración si
   necesario.</p></div>

Lectura opcional: gradientes tensoriales y productos jacobianos
 ---------------------------------------------

 En muchos casos, tenemos una función de pérdida escalar y necesitamos calcular
 el gradiente con respecto a algunos parámetros.  Sin embargo, hay casos
 cuando la función de salida es un tensor arbitrario.  En este caso, PyTorch
 le permite calcular el llamado **producto jacobiano**, y no el real
 degradado.
Para una función vectorial $\vec{y}=f(\vec{x})$, where
$\vec{x}=\langle x_1,\dots,x_n\rangle$ y
$\vec{y}=\langle y_1,\dots,y_m\rangle$,un gradiente de
$\vec{y}$ con respecto a $\vec{x}$ es dado por **Matriz Jacobina**:

\begin{align}J=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}

En lugar de calcular la propia matriz jacobiana, PyTorch le permite
computar **Producto jacobiano** $v^T\cdot J$ for a given input vector
$v=(v_1 \dots v_m)$. Esto se logra llamando ``backward()`` con
$v$ como argumento.El tamaño de $v$ debe ser el mismo que
el tamaño del tensor original, con respecto al cual queremos
calcular el producto:




In [8]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Note que cuando llamamos ``.backward()`` por segunda vez con el mismo
 argumento, el valor del gradiente es diferente.  Esto sucede porque
 al hacer la propagación ``.backward()``, PyTorch **acumula el
 gradientes**, es decir, el valor de los gradientes calculados se suma al
 Propiedad ``grad`` de todos los nodos hoja del gráfico computacional.  Si tu quieres
 para calcular los gradientes adecuados, debe poner a cero el ``grad``
 propiedad antes.  En el entrenamiento de la vida real, un *optimizador* nos ayuda a hacer
 este.


<div class="alert alert-info"><h4>Nota</h4><p>Previamente estábamos llamando a la función ``backward()`` sin
           parámetros  Esto es esencialmente equivalente a llamar
          ``backward(torch.tensor(1.0))``, que es una forma útil de calcular el
           gradientes en el caso de una función de valor escalar, como la pérdida durante
           entrenamiento de redes neuronales.</p></div>




--------------




## Veamos un ejemplo

In [24]:
import numpy as np

# f = w * x

# f = w * x
X = np.array([1, 2, 3, 4], dtype = np.float32)
Y = np.array([2, 4, 6, 8], dtype = np.float32)

w = 0.0

# model prediction
def forward(x):
    return w * x

# loss = MSE
def loss(y, y_predicted):
    return ((y_predicted - y)**2).mean()

def gradient(x, y, y_predicted):
    return np.dot(2*x, y_predicted - y).mean()

# Trainnig
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = forward(X)
    
    # loss
    l = loss(Y, y_pred)
    
    # grandients
    dw = gradient(X,Y, y_pred)
    
    #update the weights
    w -= learning_rate * dw
    
    if epoch % 2 == 0:
        print(f"epoch {epoch + 1}: w = {w:.3f}, loos = {l:.8f}")
        
print(f"Prediction after trainig: f(5) = {forward(5):.3f}")

epoch 1: w = 1.200, loos = 30.00000000
epoch 3: w = 1.872, loos = 0.76800019
epoch 5: w = 1.980, loos = 0.01966083
epoch 7: w = 1.997, loos = 0.00050331
epoch 9: w = 1.999, loos = 0.00001288
epoch 11: w = 2.000, loos = 0.00000033
epoch 13: w = 2.000, loos = 0.00000001
epoch 15: w = 2.000, loos = 0.00000000
epoch 17: w = 2.000, loos = 0.00000000
epoch 19: w = 2.000, loos = 0.00000000
Prediction after trainig: f(5) = 10.000


In [30]:
## ACTULIZANDO

# f = w * x

# f = w * x
X = torch.tensor([1, 2, 3, 4], dtype = torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype = torch.float32)

w = torch.tensor(0.0, dtype = torch.float32, requires_grad = True)

# model prediction
def forward(x):
    return w * x

# loss = MSE
def loss(y, y_predicted):
    return ((y_predicted - y)**2).mean()

# Trainnig
learning_rate = 0.01
n_iters = 20

for epoch in range(n_iters):
    # predicción = forward pass
    y_pred = forward(X)
    
    # pérdida
    l = loss(Y, y_pred)
    
    # grandients = backward() #Bueno no exatamente en parte por eso no se tienen los mismos ressultdos
    l.backward()
    
    #Actulizar los pesos
    with torch.no_grad():
        w -= learning_rate * w.grad
        
    # Poner en cero los gradientes
    w.grad.zero_()
        
    if epoch % 2 == 0:
        print(f"epoch {epoch + 1}: w = {w:.3f}, loos = {l:.8f}")
        
print(f"Prediction after trainig: f(5) = {forward(5):.3f}")

epoch 1: w = 0.300, loos = 30.00000000
epoch 3: w = 0.772, loos = 15.66018772
epoch 5: w = 1.113, loos = 8.17471695
epoch 7: w = 1.359, loos = 4.26725292
epoch 9: w = 1.537, loos = 2.22753215
epoch 11: w = 1.665, loos = 1.16278565
epoch 13: w = 1.758, loos = 0.60698116
epoch 15: w = 1.825, loos = 0.31684780
epoch 17: w = 1.874, loos = 0.16539653
epoch 19: w = 1.909, loos = 0.08633806
Prediction after trainig: f(5) = 9.612


Gracias A:
----------------
Documentacion oficial de pytorch: https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

Traducido por: Mi :)